In [1]:
import os, sys

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Embedding
from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.utils import np_utils
import re

import nltk
from nltk.corpus import stopwords
from nltk.corpus import RegexpTokenizer as regextoken

# #LOADING THE DATA

Using TensorFlow backend.


In [2]:
os.getcwd()
os.chdir('D:/INSOFE Internship/CUTE4/train')

data = pd.read_csv("train.csv",header=0)
test = pd.read_csv("test.csv",header=0)

# #UNDERSTAND THE DATA

In [3]:
data.shape 

test.shape 

data.columns

test.columns

Index(['ID', 'converse'], dtype='object')

# # DISPLAY THE INDEX

In [4]:
data.index

test.index

RangeIndex(start=0, stop=11455, step=1)

# # Understand the distribution of the data

In [5]:
data.describe(include='all')

,ID,categories,converse
count,45825.000000,45825,45796
unique,NaN,6,44465
top,NaN,PRESCRIPTION,clinical list changes medfusion secure electro...
freq,NaN,12077,133
mean,28645.024441,NaN,NaN
std,16545.373029,NaN,NaN
min,2.000000,NaN,NaN
25%,14296.000000,NaN,NaN
50%,28668.000000,NaN,NaN
75%,42965.000000,NaN,NaN


 # #DISPLAY THE DATA TYPE OF EACH VARIABLE

In [6]:
data.dtypes

ID             int64
categories    object
converse      object
dtype: object

# #Check the unique classes in categories column

In [7]:
np.unique(data['categories'])

pd.value_counts(data['categories'])

np.size(np.unique(data['categories']))

6

# # #Check the data for missing values

In [8]:
data.isnull().sum()

ID             0
categories     0
converse      29
dtype: int64

In [9]:
#Categories column is of 'object' type,changing it to category data type
data['categories'] = data['categories'].astype('category')

In [10]:
#Drop the ID column from both train and test data
data.drop('ID', axis=1, inplace=True)
test.drop('ID', axis=1, inplace=True)

In [11]:
#Check the data type after changing the data type to categorical
data.dtypes

#Check the columns 
data.columns

Index(['categories', 'converse'], dtype='object')

In [12]:
#Label Encoder encodes the categorical classes as different numbers
#Since the 'categories' column is the column to be predicted in test,set labels
#to the differone hot encoded verson
LabelEncoder = preprocessing.LabelEncoder()
labels = LabelEncoder.fit_transform(data['categories'])
y_labels= np_utils.to_categorical(labels, 6)
set(labels)
labels.shape

(45825,)

In [13]:
#Split the data into train and validation
X_train, X_validation, y_train, y_validation = train_test_split(data.iloc[:,1], y_labels, test_size=0.3, random_state=123) 

In [14]:
#Converting the X_train and X_validation is now converted to array from 
#dataframe, for ease of use convert them to list so that they can be used in 
#tokenizer

x_train_sent=[i for i in X_train]

x_validation_sent=[j for j in X_validation ]

test_sent=[k for k in test.iloc[:,0]]

In [15]:
for i in range(0,32057):
    x_train_sent[i]=re.sub("[^a-zA-Z]", " ",str(x_train_sent[i]))
    x_train_sent[i].lower()

for i in range(0,13739):
    x_validation_sent[i]=re.sub("[^a-zA-Z]", " ",str(x_validation_sent[i]))
    x_validation_sent[i].lower()

for i in range(0,11455):
    test_sent[i]=re.sub("[^a-zA-Z]", " ",str(test_sent[i]))
    test_sent[i].lower()
    
    
complete=x_train_sent+x_validation_sent+test_sent


In [16]:
# Prepare tokenizer for train and validation
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(complete)

vocab_Size = len(tokenizer1.word_index) + 1
print('Found %s unique tokens.' % vocab_Size)

Found 39289 unique tokens.


In [17]:
# integer encode the documents
#train
sequences_train = tokenizer1.texts_to_sequences(x_train_sent)


# integer encode the documents
#validation
sequences_validation = tokenizer1.texts_to_sequences(x_validation_sent)


# integer encode the documents
#test
sequences_test = tokenizer1.texts_to_sequences(test_sent)
print( sequences_test[1])

[510, 18711, 59, 8, 60, 38, 60, 296, 2, 1, 26, 73, 13, 18711, 59, 20, 98, 127, 22, 26, 553, 8, 171, 325, 111, 72, 2, 72, 1447, 1494, 22, 26, 438, 16, 2, 6, 37, 35, 707, 4, 15, 226, 569, 88, 3, 2713, 16, 4162, 2, 67, 1, 480, 15, 3, 1, 72, 157, 35283, 6, 140, 61, 6, 445, 1, 247, 282, 88, 32, 425, 3, 1169, 44, 178, 19, 1, 194, 195, 3, 103, 19, 15, 69, 6, 272, 1, 289, 327, 45, 10, 410, 7, 19, 88, 359, 3, 1, 86, 40, 6, 172, 1351, 1006, 41, 407, 18, 53, 10, 2590, 17, 524, 127, 101, 4, 3539, 7, 4, 88, 3, 18, 6, 37, 35284, 1, 4, 247, 36, 120, 3944, 30, 123, 193, 7, 171, 325, 111]


In [18]:
MAX_SEQUENCE_LENGTH = 700

train_pad = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

print('Shape of train_data tensor:', train_pad.shape)

validation_pad = pad_sequences(sequences_validation, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

print('Shape of validation_data tensor:', validation_pad.shape)

test_pad = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

print('Shape of test_data tensor:', test_pad.shape)

Shape of train_data tensor: (32077, 700)
Shape of validation_data tensor: (13748, 700)
Shape of test_data tensor: (11455, 700)


In [19]:
train_pad.shape

(32077, 700)

In [20]:
#lstm_model


# create the model


lstm_model = Sequential()
lstm_model.add(Embedding(vocab_Size, 2, input_length=MAX_SEQUENCE_LENGTH))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(100))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(6, activation='softmax'))


In [21]:
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
print(lstm_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 700, 32)           1257248   
_________________________________________________________________
dropout_1 (Dropout)          (None, 700, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 1,311,054
Trainable params: 1,311,054
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
#Fit the model
lstm_model.fit(train_pad, y_train,validation_data=(validation_pad, y_validation),epochs=4,batch_size=64,verbose=2)

Train on 32077 samples, validate on 13748 samples
Epoch 1/4
 - 816s - loss: 1.5647 - acc: 0.2520 - val_loss: 1.5441 - val_acc: 0.2671
Epoch 2/4
 - 725s - loss: 1.5546 - acc: 0.2514 - val_loss: 1.5441 - val_acc: 0.2671
Epoch 3/4
 - 637s - loss: 1.5527 - acc: 0.2573 - val_loss: 1.5441 - val_acc: 0.2671
Epoch 4/4
 - 641s - loss: 1.5514 - acc: 0.2565 - val_loss: 1.5430 - val_acc: 0.2671


In [24]:
# evaluate the model
loss, accuracy = lstm_model.evaluate(train_pad, y_train, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 26.202575


In [25]:
#Predict on validation

validation_pred = lstm_model.predict(validation_pad)
validation_pred

array([[ 0.25086784,  0.20218466,  0.00074438,  0.07854893,  0.20986401,
         0.25779021],
       [ 0.25086784,  0.20218463,  0.00074438,  0.07854892,  0.20986399,
         0.25779021],
       [ 0.25086784,  0.20218465,  0.00074438,  0.07854892,  0.20986399,
         0.25779021],
       ..., 
       [ 0.25086784,  0.20218465,  0.00074438,  0.07854892,  0.20986399,
         0.25779021],
       [ 0.25086784,  0.20218463,  0.00074438,  0.07854892,  0.20986399,
         0.25779021],
       [ 0.25086784,  0.20218465,  0.00074438,  0.07854892,  0.20986399,
         0.25779021]], dtype=float32)

In [26]:
# evaluate the model
loss, accuracy = lstm_model.evaluate(validation_pad, y_validation, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 26.709340


In [27]:
#Predict on test
test_pred = lstm_model.predict(test_pad)

In [28]:
#test_pred

test_predictions =[]
for i in test_pred:
    test_predictions.append(np.argmax(i))

In [29]:
#Inverse_transform using labelencoder

test_predictions = LabelEncoder.inverse_transform(test_predictions)


In [30]:
#test_file
test = pd.read_csv("test.csv",header=0)

copy_test=test

In [31]:
#write to the output file
output = {'ID':copy_test['ID'],'categories': test_predictions}
output_df = pd.DataFrame(data=output)

pd.value_counts(output['categories'])

output_df.to_csv('predict2.csv',index=False)